# A minimal ALF run

In this bare-bones example we use the [pyALF](https://git.physik.uni-wuerzburg.de/ALF/pyALF/-/tree/master/) interface to run the canonical Hubbard model on a default configuration: a $6\times6$ square grid, with interaction strength $U=4$ and inverse temperature $\beta = 5$.  

Bellow we go through the steps for performing the simulation and outputting observables.

---

**1.** Import `ALF_source` and `Simulation` classes from the `py_alf` python module, which provide the interface with ALF:

In [1]:
from py_alf import ALF_source, Simulation  # Interface with ALF

**2.** Create an instance of `ALF_source`, downloading the ALF source code from the [ALF repository](https://git.physik.uni-wuerzburg.de/ALF/ALF/-/tree/master/), if `alf_dir` does not exist. Gets alf_dir from environment variable `$ALF_DIR`, or defaults to "./ALF", if not present:

In [2]:
alf_src = ALF_source()

**3.** Create an instance of `Simulation`, overwriting default parameters as desired:

In [3]:
sim = Simulation(
    alf_src,
    "Hubbard",                    # Name of Hamiltonian
    {                             # Dictionary overwriting default parameters
        "Lattice_type": "Square"
    },
    machine='GNU',  # "GNU", "intel", or "PGI"
        #mpi=True,
        #n_mpi=4,
        #mpiexec='orterun',
)

**4.** Compile ALF. The first time it will also download and compile HDF5, which could take $\sim15$ minutes.

In [4]:
sim.compile()

Compiling ALF... 
Done.


**5.** Perform the simulation as specified in `sim`:

In [12]:
sim.run()

Prepare directory "/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF/Prog/ALF.out


**6.** Perform some simple analysis:

In [13]:
sim.analysis()

### Analyzing /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau


**7.** Read analysis results into a Pandas Dataframe with one row per simulation, containing parameters and observables:

In [14]:
obs = sim.get_obs()

/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square


In [15]:
obs

,continuous,ham_chem,ham_t,ham_t2,ham_tperp,ham_u,ham_u2,mz,l1,l2,...,SpinXY_tauK,SpinXY_tauK_err,SpinXY_tauR,SpinXY_tauR_err,SpinXY_tau_lattice,SpinZ_tauK,SpinZ_tauK_err,SpinZ_tauR,SpinZ_tauR_err,SpinZ_tau_lattice
/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square,0,0.0,1.0,1.0,1.0,4.0,4.0,1,6,6,...,"[[1.0580586357839297, 0.49641326376138345, 0.5...","[[0.08284902436820538, 0.10215500262022593, 0....","[[0.06375380775699882, -0.09920473127796248, 0...","[[0.007395137857331803, 0.01637199269805693, 0...","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'a1': [1....","[[0.8873384572703937, 0.6178682791301988, 0.57...","[[0.04675788230348043, 0.032378596526033594, 0...","[[0.06669130979820782, -0.0865538253191004, 0....","[[0.01890442591088348, 0.019204783465492557, 0...","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'a1': [1...."


$\bullet$ The internal energy of the system (and its error) are accessed by:

In [16]:
obs.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Ener_scal0            -29.8191
Ener_scal0_err        0.195346
Ener_scal_sign               1
Ener_scal_sign_err           0
Name: /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

$\bullet$ The simulation can be resumed by calling `sim.run()` again, increasing the precision of results:

In [17]:
sim.run()
sim.analysis()
obs2 = sim.get_obs()
obs2.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Prepare directory "/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF/Prog/ALF.out
### Analyzing /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau
/home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square


Ener_scal0             -29.852
Ener_scal0_err        0.150575
Ener_scal_sign               1
Ener_scal_sign_err           0
Name: /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

In [18]:
print("\nRunning again reduced the error from\n", obs.iloc[0][['Ener_scal0_err']], "\nto\n", obs2.iloc[0][['Ener_scal0_err']])


Running again reduced the error from
 Ener_scal0_err    0.195346
Name: /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object 
to
 Ener_scal0_err    0.150575
Name: /home/stafusa/Jobs/Uni-Wuerzburg/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object


**Note**: To run a fresh simulation - instead of performing a refinement over previous run(s) - the Monte Carlo run directory should be deleted before rerunning.

---
## Exercises
1. Check info/help commands such as `help(alf_src)`, `alf_src.get_ham_names()`, and `alf_src.get_default_params('Hubbard')`.
2. Rerun once again and check the new improvement in precision.
3. Look at a few other observables (`sim.analysis()` outputs the names of those available).
4. Change the lattice size by adding, e.g., `\"L1\": 4,` and `\"L2\": 1,` to the simulation parameters definitions of `sim` (step 2).